# Part I Review the online programming.

# Part 2: change loss function from $loss = \frac{1}{n}\sum{(y_i - \hat(y_i))^2}$ to $loss = \frac{1}{n}\sum{|y_i - \hat{y_i}|}$, and using your mathmatical knowledge to get the right partial formual. Implemen the gradient descent code.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
import functools

In [2]:
data = datasets.load_boston()

In [6]:
def hypothesis(theta, X):
    """y = theta * x + b
    """
    if len(X.shape) == 2:
        return (X * theta).sum(1)
    else:
        return (X * theta).sum()

In [8]:
def cost_func(X, y):
    """
    代价函数
    :param X:
    :param y:
    :return:
    """
    def f(theta):
        y_hat = hypothesis(theta, X)
        # 计算绝对值损失
        return np.sum(np.abs(y - y_hat))/len(y)
    return f

## 计算绝对值损失的梯度

$$
dcost() / d\theta_j =
\begin{cases}
\frac{1}{n}\sum_i^n x_j^i ,  & \text{if y >= $\hat{y}$} \\
\frac{1}{n}\sum_i^n -x_j^i, & \text{if y < $\hat{y}$}
\end{cases}
$$

In [44]:
def cost_func_gradient(X, y):
    """
    求代价函数的梯度
    :param X:
    :param y:
    :return:
    """
    def f(theta):
        m = len(X)
        partial_theta = []
        yhat = hypothesis(theta, X)
        ydiff = yhat - y
        for j, t in enumerate(theta):
            x_ij = [x[j] if ydiff[i] >= 0 else -x[j] for i,x in enumerate(X)]
            pt = np.sum(x_ij)/ m
            partial_theta.append(pt)
        return np.array(partial_theta)
    return f

In [11]:
def computeNumericalGradient(loss_func, x):
    """
    使用极限来计算梯度，用于验证导数梯度的正确性
    :param loss_func:
    :param x:
    :return:
    """
    e = 1e-4;
    if isinstance(x, np.ndarray):
        numgrad = []
        for j, t in enumerate(x):
            perturb = np.zeros(x.shape)
            perturb[j] = e
            loss1 = loss_func(x - perturb)
            loss2 = loss_func(x + perturb)
            ng = (loss2 - loss1) / (2 * e);
            numgrad.append(ng)
        return np.array(numgrad)
    else:
        loss1 = loss_func(x - e)
        loss2 = loss_func(x + e)
        ng = (loss2 - loss1) / (2 * e)
        return ng

In [18]:
def get_random_theta(l):
    """
    随机初始化一个点
    :param l:
    :return:
    """
    import random
    return np.array([random.random() * 200 - 100 for i in range(0, l)])

## 验算梯度函数是否正确

In [50]:
X, y = data['data'], data['target']
feature_num = X.shape[1]
theta = get_random_theta(feature_num)

In [51]:
cost_func_gradient(X, y)(theta)

array([-1.79968470e+00, -1.13636364e+01, -9.13361660e+00, -6.91699605e-02,
       -4.80904545e-01, -5.60395059e+00, -5.87416996e+01, -3.57086877e+00,
       -6.89328063e+00, -3.34529644e+02, -1.62199605e+01, -3.52766285e+02,
       -1.02784783e+01])

In [52]:
functools.partial(computeNumericalGradient, cost_func(X, y))(theta)

array([-1.79968472e+00, -1.13636364e+01, -9.13361657e+00, -6.91699825e-02,
       -4.80904564e-01, -5.60395058e+00, -5.87416996e+01, -3.57086878e+00,
       -6.89328068e+00, -3.34529644e+02, -1.62199605e+01, -3.52766285e+02,
       -1.02784783e+01])

In [16]:
theta = get_random_theta(feature_num)

In [17]:
cost_func(X,y)(theta)

24618.284451542306

##  梯度下降

In [53]:
def gradient_descent(start, f, df, learning_rate=0.1, max_iter=1000, accuracy = 0.001):
    # 批量梯度下降
    x = start
    num_iter = 0
    lastValue = np.inf
    trace_point = []
    while num_iter <= max_iter:
        v = f(x)
        print("num_iter = %f, v  = %f" % (num_iter, v))
        trace_point.append((x, v))
        diff = abs(v - lastValue)
        if diff <= accuracy:
            break
        lastValue = v
        g = df(x)
        x = x - learning_rate * g
        num_iter += 1
    return x, np.array(trace_point)

In [55]:
X, y = data['data'], data['target']
b = np.ones(len(X))
# 把自变量归一化一下
X = (X-X.min(0))/(X.max(0)-X.min(0))
X = np.column_stack((X,b))

In [58]:
feature_num = X.shape[1]
start_theta = get_random_theta(feature_num)
extremum_theta, traces = gradient_descent(start_theta, cost_func(X, y), functools.partial(computeNumericalGradient, cost_func(X, y)), max_iter=500, learning_rate=0.1)

num_iter = 0.000000, v  = 147.172070
num_iter = 1.000000, v  = 146.803496
num_iter = 2.000000, v  = 146.434922
num_iter = 3.000000, v  = 146.066348
num_iter = 4.000000, v  = 145.697774
num_iter = 5.000000, v  = 145.329200
num_iter = 6.000000, v  = 144.960625
num_iter = 7.000000, v  = 144.592051
num_iter = 8.000000, v  = 144.223477
num_iter = 9.000000, v  = 143.854903
num_iter = 10.000000, v  = 143.486329
num_iter = 11.000000, v  = 143.117754
num_iter = 12.000000, v  = 142.749180
num_iter = 13.000000, v  = 142.380606
num_iter = 14.000000, v  = 142.012032
num_iter = 15.000000, v  = 141.643458
num_iter = 16.000000, v  = 141.274884
num_iter = 17.000000, v  = 140.906309
num_iter = 18.000000, v  = 140.537735
num_iter = 19.000000, v  = 140.169161
num_iter = 20.000000, v  = 139.800587
num_iter = 21.000000, v  = 139.432013
num_iter = 22.000000, v  = 139.063438
num_iter = 23.000000, v  = 138.694864
num_iter = 24.000000, v  = 138.326290
num_iter = 25.000000, v  = 137.957716
num_iter = 26.000000, 

num_iter = 224.000000, v  = 64.764833
num_iter = 225.000000, v  = 64.404249
num_iter = 226.000000, v  = 64.043664
num_iter = 227.000000, v  = 63.683079
num_iter = 228.000000, v  = 63.322495
num_iter = 229.000000, v  = 62.961910
num_iter = 230.000000, v  = 62.601700
num_iter = 231.000000, v  = 62.243861
num_iter = 232.000000, v  = 61.888532
num_iter = 233.000000, v  = 61.533203
num_iter = 234.000000, v  = 61.177874
num_iter = 235.000000, v  = 60.822545
num_iter = 236.000000, v  = 60.467216
num_iter = 237.000000, v  = 60.111887
num_iter = 238.000000, v  = 59.756558
num_iter = 239.000000, v  = 59.401422
num_iter = 240.000000, v  = 59.048734
num_iter = 241.000000, v  = 58.696046
num_iter = 242.000000, v  = 58.343358
num_iter = 243.000000, v  = 57.990670
num_iter = 244.000000, v  = 57.637982
num_iter = 245.000000, v  = 57.285427
num_iter = 246.000000, v  = 56.935056
num_iter = 247.000000, v  = 56.584684
num_iter = 248.000000, v  = 56.234313
num_iter = 249.000000, v  = 55.883941
num_iter = 2

num_iter = 442.000000, v  = 23.369639
num_iter = 443.000000, v  = 23.354347
num_iter = 444.000000, v  = 23.339840
num_iter = 445.000000, v  = 23.325533
num_iter = 446.000000, v  = 23.311512
num_iter = 447.000000, v  = 23.297493
num_iter = 448.000000, v  = 23.283890
num_iter = 449.000000, v  = 23.270346
num_iter = 450.000000, v  = 23.257146
num_iter = 451.000000, v  = 23.243946
num_iter = 452.000000, v  = 23.230960
num_iter = 453.000000, v  = 23.218195
num_iter = 454.000000, v  = 23.205548
num_iter = 455.000000, v  = 23.193791
num_iter = 456.000000, v  = 23.182406
num_iter = 457.000000, v  = 23.171022
num_iter = 458.000000, v  = 23.159666
num_iter = 459.000000, v  = 23.148757
num_iter = 460.000000, v  = 23.138230
num_iter = 461.000000, v  = 23.127720
num_iter = 462.000000, v  = 23.117577
num_iter = 463.000000, v  = 23.107433
num_iter = 464.000000, v  = 23.097296
num_iter = 465.000000, v  = 23.087520
num_iter = 466.000000, v  = 23.077743
num_iter = 467.000000, v  = 23.067967
num_iter = 4

## 求解的参数为

In [59]:
extremum_theta

array([-20.0949827 , -26.96855248, -31.77713404, -43.41337301,
        22.80520772,  -8.37433782,  68.67624719, -37.18459886,
        65.13613504, -78.70718032, -21.35563735,  50.67117382,
       -78.52977709,  -7.93790248])

# Part 3: Finish the Solution Parse Part of Edit-Distance

In [34]:
from functools import lru_cache

@lru_cache(maxsize=2 ** 10)
def edit_distance_func():
    """

    :param string1:
    :param string2:
    :return:
    """
    solution = {}
    def f(string1, string2):
#         print(string1, "<-->", string2)

        if string1 == string2:
            solution[(string1, string2)] = "EQUAL"
            return (0, "EQUAL")

        # 如果其中一个长度为0，那么需要的步数就是另一个的长度数，等于全部都是新增N个字母以达到另一个的长度
        if len(string1) == 0: return (len(string2), "ZERO")
        if len(string2) == 0: return (len(string1), "ZERO")

        tail_s1 = string1[-1]
        tail_s2 = string2[-1]

        # 尝试三种方法，ADD， DEL， SUB，然后看哪一种步数最少
        del_step = f(string1[:-1], string2)
        add_step = f(string1, string2[:-1])
        candidates = [
            # 如果左边删除一个得到右边，那么需要的步数就是左边去掉末尾的字符串变到右边的步数加一
            (del_step[0] + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
            # 如果右边删除一个能得到左边，那么需要的步数就是左边变到右边去掉末尾的字符串的步数加一
            (add_step[0] + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
        ]

        # 尝试替换
        if tail_s1 == tail_s2:
            both_forward = (f(string1[:-1], string2[:-1])[0] + 0, 'BOTH_FORWARD')
        else:
            # 如果最后字母不一样，相当于最后一步需要替换成一样，然还再递归看其字串的步数
            both_forward = (f(string1[:-1], string2[:-1])[0] + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

        candidates.append(both_forward)

        # 找出最短的
        min_distance, operation = min(candidates, key=lambda x: x[0])
        # 记录下来方案
        solution[(string1, string2)] = operation
        print("add solution", string1, "<-->", string2, " : ", operation)
        return (min_distance, solution)

    return f


In [16]:
edit_distance = edit_distance_func()

In [19]:
min_distance, solution = edit_distance('ABCDE', 'ABCCEF')

In [20]:
min_distance

2

In [35]:
edit_distance('biejin', 'beijing')

(3,
 {('A', 'A'): 'EQUAL',
  ('A', 'AB'): 'ADD B',
  ('A', 'ABC'): 'ADD C',
  ('A', 'ABCC'): 'ADD C',
  ('A', 'ABCCE'): 'ADD E',
  ('A', 'ABCCEF'): 'ADD F',
  ('AB', 'AB'): 'EQUAL',
  ('AB', 'ABC'): 'ADD C',
  ('AB', 'ABCC'): 'ADD C',
  ('AB', 'ABCCE'): 'ADD E',
  ('AB', 'ABCCEF'): 'ADD F',
  ('ABC', 'ABC'): 'EQUAL',
  ('ABC', 'ABCC'): 'ADD C',
  ('ABC', 'ABCCE'): 'ADD E',
  ('ABC', 'ABCCEF'): 'ADD F',
  ('AB', 'A'): 'DEL B',
  ('ABC', 'A'): 'DEL C',
  ('ABC', 'AB'): 'DEL C',
  ('ABCD', 'A'): 'DEL D',
  ('ABCD', 'AB'): 'DEL D',
  ('ABCD', 'ABC'): 'DEL D',
  ('ABCD', 'ABCC'): 'SUB D => C',
  ('ABCD', 'ABCCE'): 'ADD E',
  ('ABCD', 'ABCCEF'): 'ADD F',
  ('ABCDE', 'A'): 'DEL E',
  ('ABCDE', 'AB'): 'DEL E',
  ('ABCDE', 'ABC'): 'DEL E',
  ('ABCDE', 'ABCC'): 'DEL E',
  ('ABCDE', 'ABCCE'): 'BOTH_FORWARD',
  ('ABCDE', 'ABCCEF'): 'ADD F',
  ('A', 'AT'): 'ADD T',
  ('A', 'ATC'): 'ADD C',
  ('A', 'ATCG'): 'ADD G',
  ('A', 'ATCGG'): 'ADD G',
  ('A', 'ATCGGG'): 'ADD G',
  ('A', 'ATCGGGA'): 'ADD A',


In [36]:
def find_solution(s1, s2):
    def f(string1, string2, solution, cur_string = None, index = -1):
        if not cur_string: cur_string = s1
        s = solution[(string1, string2)]
        if s == "EQUAL":
            return
        elif s.startswith("ADD"):
            new_cur_string = cur_string[:index + 1] + string2[-1] + cur_string[index+1:]
            print(cur_string, " ",s, " --> ", new_cur_string)
            return f(string1, string2[:-1], solution, new_cur_string, index)
        elif s.startswith("DEL"):
            new_cur_string = cur_string[:index] + cur_string[index+1:]
            print(cur_string, " ",s, " --> ", new_cur_string)
            return f(string1[:-1], string2, solution, new_cur_string, index-1)
        elif s.startswith("SUB"):
            new_cur_string = cur_string[:index]  + string2[-1] + cur_string[index + 1:]
            print(cur_string, " ",s, " --> ", new_cur_string)
            return f(string1[:-1], string2[:-1], solution, new_cur_string, index-1)
        elif s.startswith("BOTH_FORWARD"):
            return f(string1[:-1], string2[:-1], solution, cur_string, index - 1)

    min_distance, solution = edit_distance(s1, s2)
    print(s1, " --> ", s2, " distance is ", min_distance)
    return f(s1, s2, solution, index=len(s1)-1)

In [37]:
find_solution('ABCDE', 'ABCCEF')

ABCDE  -->  ABCCEF  distance is  2
ABCDE   ADD F  -->  ABCDEF
ABCDEF   SUB D => C  -->  ABCCEF


In [38]:
find_solution("ABC", "ABCCEF")

ABC  -->  ABCCEF  distance is  3
ABC   ADD F  -->  ABCF
ABCF   ADD E  -->  ABCEF
ABCEF   ADD C  -->  ABCCEF


In [39]:
find_solution('ATCGGAA', 'ATCGGGA')

ATCGGAA  -->  ATCGGGA  distance is  1
ATCGGAA   SUB A => G  -->  ATCGGGA


In [40]:
find_solution('biejin', 'beijing')

biejin  -->  beijing  distance is  3
biejin   ADD g  -->  biejing
biejing   DEL e  -->  bijing
bijing   ADD e  -->  beijing


# Part 4 Choose 1 - 2 books to keep reading

# Part 5-1: review machine learning


### Why do we use Derivative / Gredient to fit a target function?
Ans: 在方程没有或很难求得解析解的情况下，梯度下降可以求得一个近似的数值解

### In the words 'Gredient Descent', what's the Gredient and what's the Descent?¶
Ans: 梯度即各个自变量的偏导数组成的偏导向量，梯度的方向即函数增长最快的方向，反之，梯度的反方向即是函数值下降最快的方向。

###  What's the advantages of the 3rd gradient descent method compared to the previous methods?
Ans: 梯度下降能够精确的知道当前位置函数值减少最快的方向，不用盲目的去做无用的尝试，可以很快的找到损失函数的极值点。

### Using the simple words to describe: What's the machine leanring.¶
Ans:机器学习就是让机器从数据中学习一种方法以解决特定的问题，并且随着数据的增多，机器学到的方法的性能也随之提高。

# Part 5: Answer following questions:

### Why do we need dynamic programming? What's the difference of dynamic programming and previous talked search problme?
动态规划是把问题分解为更细的问题，一边搜索并且一边保存子问题的解的一种方法。 而之前的 BFS 或者 DFS 是直接搜索需要的解并没有分解为子问题并保存中间的结果。


### Why do we still need dynamic programming? Why not we train a machine learning to fit a function which could get the right answer based on inputs?
根据“没有免费的午餐”定理，机器学习并不能解决所有的问题，不管什么问题都套用机器学习的方法有时只会南辕北辙事倍功半。


### Can you catch up at least 3 problems which could solved by Dynamic Programming?
1 计算城市间的距离，可以分解各个城市的距离的和。
2 最长公共子序列
3 最长递增子序列


### Can you catch up at least 3 problems wich could sloved by Edit Distance?
1 拼写检查
2 近似字符串匹配
3 语音识别效果评测

### Please summarize the three main features of Dynamic Programming, and make a concise explain for each feature.

1 重复子问题
    问题可以分解为多个子问题，子问题的求解过程中又包含重复的子问题。
2 可以保存中间重复子问题的解
3 解析存储的结果
根据已经存储的子问题的解得到大问题的解

### What's the disadvantages of Dynamic Programming? (You may need search by yourself in Internet)
1. 没有统一的标准模型；
2. 数值方法求解时存在维数灾。